In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from neuro import config
from os.path import join
sys.path.append(join(config.REPO_DIR, 'experiments'))

import dvu
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
import numpy as np
from neuro import config
import imodelsx.process_results
import neuro.features.qa_questions as qa_questions
import joblib
from tqdm import tqdm
import neuro.viz
from neuro import analyze_helper, viz
fit_encoding = __import__('02_fit_encoding')
dvu.set_style()

results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jul24_2025_agentic_test'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
# joblib.dump({'r': rr, 'cols_varied': cols_varied, 'mets': mets}, 'results_best_ensemble.pkl')
# data = joblib.load(join(config.RESULTS_DIR_LOCAL, 'results_best_ensemble.pkl'))
# rr, cols_varied, mets = data['r'], data['cols_varied'], data['mets']
metric_sort = 'corrs_tune_pc_weighted_mean'

FileNotFoundError: [Errno 2] No such file or directory: '/home/chansingh/mntv1/deep-fMRI/encoding/jul24_2025_agentic_test'